In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import torchvision
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import os
import cv2
from torch.utils.data import Dataset
from PIL import Image

/usr/local/home/sgchr/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
def get_handler(name):
    if name == 'MNIST':
        return DataHandler3
    elif name == 'FashionMNIST':
        return DataHandler3
    elif name == 'BreaKHis':
        return DataHandler3


class DataHandler3(Dataset):
    def __init__(self, X, Y,transform = None):  #last argument is transform= None
        self.X = X
        self.Y = Y
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        if self.transform is not None:
            x = self.transform(x)
            x = x.permute(1, 0, 2)
        return x, y, index

    def __len__(self):
        return len(self.X)
handler = get_handler('BreaKHis')    

In [35]:

original_tensor = torch.randn(32, 3, 32)

# Permute dimensions to get the desired shape (3, 32, 32)
desired_shape_tensor = original_tensor.permute(1, 2, 0)

# Display the shape of the resulting tensor
print(desired_shape_tensor.shape)


torch.Size([3, 32, 32])


In [3]:
train_dataset=torchvision.datasets.ImageFolder('/usr/local/home/sgchr/Documents/Cancer_classification/BreaKHis_v1/Cancer_train',transform=transforms.Compose([transforms.ToTensor(),transforms.RandomResizedCrop(32), transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip()]))
test_dataset=torchvision.datasets.ImageFolder('/usr/local/home/sgchr/Documents/Cancer_classification/BreaKHis_v1/Cancer_test',transform=transforms.Compose([transforms.ToTensor(),transforms.RandomResizedCrop(32)]))


In [57]:
cancer_X_tr = train_dataset.imgs
cancer_Y_tr = train_dataset.targets
# cancer_X_te = test_dataset.data
cancer_Y_te = test_dataset.targets

In [58]:
print(cancer_X_tr[:3])

[('/usr/local/home/sgchr/Documents/Cancer_classification/BreaKHis_v1/Cancer_train/benign/image4407.png', 0), ('/usr/local/home/sgchr/Documents/Cancer_classification/BreaKHis_v1/Cancer_train/benign/image4408.png', 0), ('/usr/local/home/sgchr/Documents/Cancer_classification/BreaKHis_v1/Cancer_train/benign/image4409.png', 0)]


In [50]:
data_tr = torchvision.datasets.CIFAR10('../data' + '/CIFAR10', train=True, download=True)
data_te = torchvision.datasets.CIFAR10('../data' + '/CIFAR10', train=False, download=True)

100%|██████████| 170498071/170498071 [00:15<00:00, 10722821.05it/s]


Extracting ../data/CIFAR10/cifar-10-python.tar.gz to ../data/CIFAR10
Files already downloaded and verified


In [51]:
CIFAR_X_tr = data_tr.data
CIFAR_Y_tr = data_tr.targets
CIFAR_X_te = data_te.data
CIFAR_Y_te = data_te.targets

In [5]:
def get_dataset():
 

    loader_tr = DataLoader(train_dataset, batch_size=1000)
    loader_te = DataLoader(test_dataset, batch_size=1000)
    X_tr, Y_tr, X_te, Y_te = [],[],[],[]
    for (x,y) in loader_tr:
        X_tr.append(x), Y_tr.append(y)

    X_tr = torch.cat(X_tr, dim=0)
    X_tr = X_tr.permute(0, 2, 3, 1)
    X_tr = X_tr.numpy()

    Y_tr = torch.cat(Y_tr, dim=0)

    # X_tr , Y_tr= np.array(X_tr), np.array(Y_tr)

    for (x,y) in loader_te:
        X_te.append(x), Y_te.append(y)

    X_te = torch.cat(X_te, dim=0)
    X_te = X_te.permute(0, 2, 3, 1)
    X_te = X_te.numpy()

    Y_te = torch.cat(Y_te, dim=0)

    return X_tr, X_te, Y_tr, Y_te
X_tr, X_te, Y_tr, Y_te = get_dataset()

In [29]:
# Assuming you have your dataset (X, Y) and a transform function (transform_fn)
test_data = DataHandler3(X_te, Y_te)
train_data = DataHandler3(X_tr, Y_tr)

In [30]:
for x, y, idx in test_data:
    # Process or use x, y, idx as needed
    print(f"Index: {idx}, Label: {y}")
    break


Index: 0, Label: 0


In [10]:
for x_train, y_train, idx in train_data:
    # Process or use x, y, idx as needed
    print(f"Index: {idx}, Label: {y}")
    break

Index: 0, Label: 0


In [31]:
args = {'n_epoch': 20, 'transform':transforms.Compose([
                    transforms.RandomRotation(20),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=(0.2, 0.2)),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean and std
                ]),
                'loader_tr_args':{'batch_size': 64, 'num_workers': 1},
                'loader_te_args':{'batch_size': 64, 'num_workers': 1},
                'optimizer_args':{'lr': 0.00001, 'momentum': 0.5},
                'transformTest' : transforms.Compose([transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
                }

In [45]:
loader_tr = DataLoader(handler(X_tr, torch.Tensor(Y_tr.numpy()).long(), transform = args['transform']), shuffle=True, **args['loader_tr_args'])
loader_te = DataLoader(handler(X_te, torch.Tensor(Y_te.numpy()).long(), transform = args['transform']), shuffle=True, **args['loader_te_args'])

In [46]:
for batch_idx, (x, y, idxs) in enumerate(loader_tr):
    print(x.shape)
    break


torch.Size([64, 3, 32, 32])


In [47]:
for batch_idx, (x_te, y_te, idxs) in enumerate(loader_te):
    print(x_te.shape)
    break

torch.Size([64, 3, 32, 32])


In [35]:
count = 0
for batch_idx, (images, labels) in enumerate(loader_tr):
    # 'images' is a tensor containing the images in the batch
    # 'labels' is a tensor containing the corresponding labels
    count+=1
    # Your training loop or processing here
    print('Batch Index:', batch_idx)
    print('Images Shape:', images.shape)
    print('Labels Shape:', labels.shape)
    if count == 10:
        break

Batch Index: 0
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 1
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 2
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 3
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 4
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 5
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 6
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 7
Images Shape: torch.Size([1000, 3, 32, 32])
Labels Shape: torch.Size([1000])
Batch Index: 8
Images Shape: torch.Size([812, 3, 32, 32])
Labels Shape: torch.Size([812])


In [7]:
Y_tr

tensor([0, 0, 0,  ..., 1, 1, 1])

In [8]:
X_tr = torch.cat(X_tr, dim=0)
X_tr = X_tr.numpy()
Y_tr = torch.cat(Y_tr, dim=0)
# X_tr , Y_tr= np.array(X_tr), np.array(Y_tr)

X_te = torch.cat(X_te, dim=0)
X_te = X_te.numpy()
Y_te = torch.cat(Y_te, dim=0)
# X_te , Y_te= np.array(X_te), np.array(Y_te)

In [7]:
import torch.nn as nn
import torch.nn.functional as F
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class CancerModel(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(CancerModel, self).__init__()
        self.in_planes = 16
        self.embDim = 128 * block.expansion
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2)
        self.linear = nn.Linear(128 * block.expansion, num_classes, bias=False)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        emb = out.view(out.size(0), -1)
        out = self.linear(emb)
        return out, emb
    def get_embedding_dim(self):
        return self.embDim    

def Cancer(num_classes=2):
    return CancerModel(BasicBlock, [2,2,2,2], num_classes)

net = Cancer(num_classes=2)

In [22]:
handler = get_handler('BreaKHis')
train_loader = DataLoader(handler(X_train, Y_train, transform=args['BreaKHis']['transform']), shuffle=False, **args['BreaKHis']['loader_te_args'])


In [24]:
train_loader = DataLoader(handler(train_dataset, test_dataset, transform=args['BreaKHis']['transform']), shuffle=False, **args['BreaKHis']['loader_te_args'])


In [43]:
P = torch.zeros(len(loader_tr.dataset)).long()

for batch_idx, (images, labels) in enumerate(loader_tr):
    batch_size = len(labels)
    indices = batch_idx * loader_tr.batch_size + torch.arange(batch_size)
    out, e1 = net(images)
    _, preds = torch.max(out, 1)
    pred = out.max(1)[1]
    P[indices] = pred.data.cpu()

accur = 1.0 * (labels == P[:len(labels)]).sum().item() / len(labels)


In [44]:
P = torch.zeros(len(loader_tr.dataset)).long()
all_labels = torch.zeros(len(loader_tr.dataset)).long()

for batch_idx, (images, labels) in enumerate(loader_tr):
    batch_size = len(labels)
    indices = batch_idx * loader_tr.batch_size + torch.arange(batch_size)
    out, e1 = net(images)
    _, preds = torch.max(out, 1)
    pred = out.max(1)[1]
    P[indices] = pred.data.cpu()
    all_labels[indices] = labels

accur = 1.0 * (all_labels == P).sum().item() / len(all_labels)


In [14]:
for (x,y) in loader_tr:
    print(x.dtype)
    break

torch.float32


In [6]:
Y_tr[6].shape

torch.Size([1000])

In [8]:
print(X_tr[0].shape)

torch.Size([1000, 3, 32, 32])
